In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Grounding in Vertex AI

**Note**: このノートブックは、[Generative AI](https://github.com/GoogleCloudPlatform/generative-ai) リポジトリの [Vertex AI での Grounding の開始方法](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/grounding/intro-grounding.ipynb) ノートブックから改変したもので、Google Cloud Skills Boost 演習のコンテキスト内で実行されるように設計されています。

## 概要

[Vertex AI での Grounding](https://cloud.google.com/vertex-ai/docs/generative-ai/grounding/ground-language-models) を使用すると、言語モデル (例: [`gemini-1.5-pro`](https://cloud.google.com/vertex-ai/docs/generative-ai/language-model-overview)) を使用して、独自のドキュメントとデータに基づいたコンテンツを生成できます。この機能により、モデルは実行時にトレーニング データを超える情報にアクセスできます。[Vertex AI Agent Builder](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction) 内のモデル応答データ ストアをグラウンディングすることで、データにグラウンディングされた LLM は、より正確で最新の関連性の高い応答を生成できます。

グラウンディングには次の利点があります:

- モデルの幻覚 (モデルが事実に基づかないコンテンツを生成するインスタンス) を削減します
- モデルの応答を特定の情報、ドキュメント、およびデータ ソースに固定します
- 生成されたコンテンツの信頼性、精度、および適用性を強化します

Vertex AI のグラウンディングのコンテキストでは、[Vertex AI Agent Builder のデータ ストア](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest) を構成できます。これには、ウェブサイト データ、非構造化データ、または構造化データの形式で独自のデータを含めることができます

## 始める前に

このノートブックは、[Vertex AI Workbench インスタンス](https://cloud.google.com/vertex-ai/docs/workbench/instances/introduction) 環境で実行するように作成されています。

### インストール

このノートブックを実行するには、必要な `google-cloud-aiplatform` パッケージをインストールします。

In [ ]:
!pip install --upgrade --quiet google-cloud-aiplatform

パッケージをインストールした後、カーネルを再起動します。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Note**: 上記のセルを実行すると、「`intro-grounding-ql.ipynb のカーネルが停止したようです。自動的に再起動します`」というメッセージが表示されます。これはカーネルの再起動による意図された動作です。次のセルでノートブックの作業を続行できます。

### プロジェクトIDとリージョンを設定する

In [ ]:
# This function retrieves the project ID value set in the Workbench Instance environment.
def get_project_id():
    project_list = !gcloud config get-value project 
    return project_list[0]

PROJECT_ID = get_project_id()
REGION = "us-central1"

print(f'Project ID: {PROJECT_ID}')

### Import libraries

In [ ]:
from IPython.display import Markdown, display
import vertexai
from vertexai.generative_models import (
    GenerationResponse,
    GenerativeModel,
    Tool,
    grounding,
)
from vertexai.preview.generative_models import grounding as preview_grounding

### 接地された Gemini 出力の可読性を向上させるヘルパー関数

In [ ]:
def print_grounding_response(response: GenerationResponse):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    for grounding_support in grounding_metadata.grounding_supports:
        text_segment = text_bytes[
            prev_index : grounding_support.segment.end_index
        ].decode(ENCODING)

        footnotes_text = ""
        for grounding_chunk_index in grounding_support.grounding_chunk_indices:
            footnotes_text += f"[{grounding_chunk_index + 1}]"

        markdown_text += f"{text_segment} {footnotes_text}\n"
        prev_index = grounding_support.segment.end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n----\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_text += f"\n**Search Entry Point:**\n {grounding_metadata.search_entry_point.rendered_content}\n"
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    markdown_text += "### Grounding Chunks\n"

    for index, grounding_chunk in enumerate(
        grounding_metadata.grounding_chunks, start=1
    ):
        context = grounding_chunk.web or grounding_chunk.retrieved_context
        if not context:
            print(f"Skipping Grounding Chunk {grounding_chunk}")
            continue

        markdown_text += f"{index}. [{context.title}]({context.uri})\n"

    display(Markdown(markdown_text))

### Vertex AI SDK と Vertex AI からの生成モデルを初期化する

In [ ]:
vertexai.init(project=PROJECT_ID, location=REGION)

model = GenerativeModel("gemini-1.5-pro")

# 例: カスタム ドキュメントとデータによるグラウンディング

この例では、グラウンディングなしの LLM レスポンスと、[Vertex AI Search のデータストアの結果](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest) でグラウンディングされたレスポンスを比較します。BigQuery クエリ内で Gemini を使用する方法について質問します。

**`DATA_STORE_ID` 変数の値は、ラボで先ほど作成したデータストアの ID に置き換えてください。**

In [ ]:
DATA_STORE_ID = "google-cloud-website_<some_numbers>"  # Replace this with your data store ID from Vertex AI Search!
DATA_STORE_REGION = "global"

ここで、BigQuery のオブジェクト テーブルとその使用タイミングについて質問してみましょう。

### グラウンディングなしのテキスト生成

グラウンディングなしで LLM に予測リクエストを送信します。

In [ ]:
PROMPT = "BigQuery SQL クエリで Gemini を使用するにはどうすればよいですか?"

In [ ]:
response = model.generate_content(PROMPT)

display(Markdown(response.text))

### Vertex AI 検索結果に基づいたテキスト生成

これで、`tools` キーワード引数に `grounding.VertexAISearch()` というグラウンディング ツールを追加して、最初にカスタム データ ストア内で検索を実行し、次に関連ドキュメントに基づいて回答を作成するように LLM に指示できます。

In [ ]:
tool = Tool.from_retrieval(
    preview_grounding.Retrieval(
        preview_grounding.VertexAISearch(
            datastore=DATA_STORE_ID,
            project=PROJECT_ID,
            location=DATA_STORE_REGION,
        )
    )
)

response = model.generate_content(
    PROMPT,
    tools=[tool],
)

print_grounding_response(response)

根拠のない応答には、BigQuery での Gemini の使用に関する LLM からの限られた情報しか含まれておらず、正確ではない可能性があることに注意してください。一方、Vertex AI Search の結果に基づいた応答には、BigQuery に関する Google Cloud ドキュメントからの最新情報が含まれています。

## 例: グラウンディングされたチャット応答

Vertex AI でチャット モデルを操作するときにもグラウンディングを使用できます。この例では、グラウンディングのない LLM 応答と、Google 検索の結果と Vertex AI Search のデータ ストアにグラウンディングされた応答を比較します。

Vertex AI に関する質問をし、続いて Vertex AI で管理されているデータセットに関する質問をします。

In [ ]:
PROMPT = "Vertex AI の管理対象データセットとは何ですか?"
PROMPT_FOLLOWUP = "どのようなタイプのデータを使用できますか"

In [ ]:
chat = model.start_chat()

response = chat.send_message(PROMPT)
display(Markdown(response.text))

response = chat.send_message(PROMPT_FOLLOWUP)
display(Markdown(response.text))

### Vertex AI 検索結果に基づいたチャット セッション

これで、`grounding_source` キーワード引数に `GroundingSource.VertexAISearch()` のグラウンディング ソースを追加して、チャット モデルに最初にカスタム データ ストア内で検索を実行し、次に関連ドキュメントに基づいて回答を作成するように指示できます。

In [ ]:
chat = model.start_chat()
tool = Tool.from_retrieval(
    preview_grounding.Retrieval(
        preview_grounding.VertexAISearch(
            datastore=DATA_STORE_ID,
            project=PROJECT_ID,
            location=DATA_STORE_REGION,
        )
    )
)
response = chat.send_message(PROMPT, tools=[tool])
print_grounding_response(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
print_grounding_response(response)